In [1]:
"""
Feature Encoding Script
@jjia
"""

'\nFeature Encoding Script\n@jjia\n'

In [2]:
import os, re, sys
import numpy as np
import pandas as pd
from math import log
import random
import scipy.io
from collections import defaultdict

In [3]:
# in a well-defined initial state.
random.seed
np.random.seed(1337)

In [4]:
def read_blosum(path,one_hot):
    '''
    Read the blosum matrix from the file blosum50.txt
    Args:
        1. path: path to the file blosum50.txt
    Return values:
        1. The blosum50 matrix
    '''
    f = open(path,"r")
    blosum = []
    if one_hot ==0: #(blosum 50)
       for line in f:
           blosum.append([(float(i))/10 for i in re.split("\t",line)])
    else:
        for line in f: #(one-hot)
           blosum.append([float(i) for i in re.split("\t",line)])
    f.close()
    return blosum

path_dict = "/home/jjia1/viralepitope/"
blosum_matrix = read_blosum(path_dict + 'blosum50.txt', 0)

pseq_dict = np.load(path_dict + 'pseq_dict_all.npy', allow_pickle = True).item()
#pseq_dict_blosum_matrix = pseudo_seq(pseq_dict, blosum_matrix)


In [5]:
np.save('../viralepitope/blosum_matrix.npy', blosum_matrix)

In [6]:
pos_data = pd.read_csv('positive_epitope_seq.txt', sep = '\t', header = None)
neg_data = pd.read_csv('negative_epitope_seq.txt', sep = '\t', header = None)

In [7]:
pos_data

,0,1,2
0,KLEDLERDL,HLA-A*02:01,1
1,LITGRLQSL,HLA-A2,1
2,EVMPVSMAK,HLA-A*03:01,1
3,EVMPVSMAK,HLA-A*11:01,1
4,KTFPPTEPK,HLA-A*03:01,1
...,...,...,...
88035,ILLWQPIPV,HLA-A*02:01,1
88036,ALDVYNGLL,HLA-A*02:01,1
88037,LTDAVKVMDL,HLA-A*02:01,1
88038,KLQCVDLHV,HLA-A*02:01,1


In [8]:
neg_data

,0,1,2
0,TDFEKAHRS,HLA-A2,0
1,FESKYSFHP,HLA-A2,0
2,GWYPKPLTV,HLA-A2,0
3,IHTGVKPYECK,HLA-A2,0
4,QDQFDMVSD,HLA-A2,0
...,...,...,...
880385,LQLPGFCLP,HLA-A*68:12,0
880386,KEKSCVWCR,HLA-A*68:12,0
880387,DSKLPAELAT,HLA-A*68:12,0
880388,KASLMNDDF,HLA-A*68:12,0


In [9]:
data = pd.concat([pos_data, neg_data], axis = 0)

In [10]:
data_shuffled = data.sample(frac =1)

In [11]:
data_shuffled.reset_index(drop = True)

,0,1,2
0,EQPMTGLGI,HLA-A*33:01,0
1,MHLFVSFILR,HLA-A*02:01,0
2,FLTIVLIIVS,HLA-B*51:01,0
3,ALEREQDALW,HLA-A*11:01,0
4,HSYGIDLKSY,HLA-A*01:01,1
...,...,...,...
968425,NQMAEPLPP,HLA-B*35:01,0
968426,IRANLVCRR,HLA-A*02:06,0
968427,RRLLLREPR,HLA-A*02:01,0
968428,ITGQSLGYG,HLA-A*30:02,0


In [12]:
data_shuffled.to_csv('fulldata_withlabels_combined.txt', sep = '\t', header = False, index = False)

In [13]:
x = pseq_dict
print(x.get('HLA-C*05:01'))
print(len(x.get('HLA-C*05:01')))

YYAGYREKYRQTDVNKLYLRYNFYTWAERAYTWY
34


In [14]:
print(x.get('HLA-B*35:01'))
print(len(x.get('HLA-B*35:01')))

YYATYRNIFTNTYESNLYIRYDSYTWAVLAYLWY
34


In [15]:
len(pseq_dict)

10387

In [16]:
def convert_alleleseq_to_pseudoseq(seq_dict, blosum_matrix):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}

    residue_indices = list(range(0,34,1))
    #Now encode the MHC sequences into pseudo-sequences.
    for allele in seq_dict.keys():
        new_pseq = []
        pseq =""
        for index in residue_indices:
            pseq+= seq_dict[allele][index]
            new_pseq.append(blosum_matrix[aa[seq_dict[allele][index]]]) 
        pseq_dict[allele] = new_pseq
    
    return pseq_dict

pseq_dict_converted = convert_alleleseq_to_pseudoseq(pseq_dict, blosum_matrix)

In [17]:
len(pseq_dict_converted)

10387

In [18]:
def mhc_peptide_pair(path, pseq_dict_matrix, blosum_matrix):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}
    data_dict = {}
    pep_length = [8,9,10,11,12,13,14,15]
    f = open(path,"r")
    for line in f:
        info = re.split("\t",line)#Retrive information from a tab-delimited line
        allele = info[1].strip()
        if allele in pseq_dict.keys():
            affinity = int(info[-1].strip()) #Retrive lable information 
            pep = info[0].strip() #Retrive ligand information 
            
            if set(list(pep)).difference(list('ACDEFGHIKLMNPQRSTVWY')):
                print('Illegal peptides')
                continue   
            if len(pep) not in pep_length:
                print('Illegal peptides')
                continue 
                
            pep_blosum = []#Encoded peptide seuqence
            for residue_index in range(15):
                #Encode the peptide sequence in the 1-12 columns, with the N-terminal aligned to the left end
                #If the peptide is shorter than 12 residues, the remaining positions on
                #the rightare filled will zero-padding
                if residue_index < len(pep):
                    pep_blosum.append(blosum_matrix[aa[pep[residue_index]]])
                else:
                    pep_blosum.append(np.zeros(20))
            for residue_index in range(15):
                #Encode the peptide sequence in the 13-24 columns, with the C-terminal aligned to the right end
                #If the peptide is shorter than 12 residues, the remaining positions on
                #the left are filled will zero-padding
                if 15 - residue_index > len(pep):
                    pep_blosum.append(np.zeros(20)) 
                else:
                    pep_blosum.append(blosum_matrix[aa[pep[len(pep) - 15 + residue_index]]])

            new_data = [pep_blosum, pseq_dict_matrix[allele], affinity]
            
            if allele not in data_dict.keys():
                data_dict[allele] = [new_data]
            else:
                data_dict[allele].append(new_data)
                
    return data_dict

In [19]:
##pos_pseudo_seq = mhc_peptide_pair('viralepitope/iedb_data.txt', pseq_dict, blosum_matrix)
#type(pos_pseudo_seq)
#neg_pseudo_seq = mhc_peptide_pair('viralepitope/negative_epitope_seq_affinity.txt', pseq_dict, blosum_matrix)
#neg_pseudo_seq['HLA-A*24:03']

In [20]:
pseudo_seq = mhc_peptide_pair('fulldata_withlabels_combined.txt', pseq_dict, blosum_matrix)

Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides


In [21]:
pseudo_seq_converted = mhc_peptide_pair('fulldata_withlabels_combined.txt', pseq_dict_converted, blosum_matrix)

Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides


In [22]:
len(pseudo_seq)

121

In [23]:
len(pseudo_seq_converted)

121

In [24]:
import pickle

In [25]:
with open('fulldata_pseudoseq_dictionary.pkl', 'wb') as f:
    pickle.dump(pseudo_seq,f)

In [26]:
with open('fulldata_pseudoseq_dictionary_converted.pkl', 'wb') as f:
    pickle.dump(pseudo_seq_converted, f)

In [27]:
#with open('pos_pseudo_seq_dictionary.pkl', 'wb') as f:
#    pickle.dump(pos_pseudo_seq, f)
#with open('neg_pseudo_seq_dictionary.pkl', 'wb') as f:
#    pickle.dump(neg_pseudo_seq, f)

In [28]:
#import csv
# open file for writing, "w" is writing
#w = csv.writer(open("viralepitope/positive_pseudo_seq.csv", "w"))

# loop over dictionary keys and values
#for key, val in pos_pseudo_seq.items():
#    # write every key and value to file
#    w.writerow([key, val])

In [29]:
# open file for writing, "w" is writing
#w1 = csv.writer(open("viralepitope/negative_pseudo_seq.csv", "w"))

# loop over dictionary keys and values
#for key, val in neg_pseudo_seq.items():
    # write every key and value to file
#    w1.writerow([key, val])